In [1]:
import pandas as pd
import json

In [2]:
summary = pd.read_csv("summary.csv")
stations = pd.read_csv("stations.csv")

In [3]:
summary.sort_values(['start', 'trips'], ascending=[True,False], inplace=True)

In [4]:
summary.reset_index(drop=True, inplace=True)

In [5]:
summary.head()

,start,end,trips
0,10th & E St NW,10th & E St NW,161
1,10th & E St NW,2nd & G St NE,69
2,10th & E St NW,1st & M St NE,60
3,10th & E St NW,1st & K St NE,56
4,10th & E St NW,1st & K St SE,56


In [6]:
stations.head()

,station_name,station_id,lat,lng
0,Constitution Ave & 2nd St NW/DOL,305.0,38.892275,-77.013917
1,8th & O St NW,413.0,38.908640,-77.022770
2,Massachusetts Ave & Dupont Circle NW,24.0,38.910100,-77.044400
3,Henry Bacon Dr & Lincoln Memorial Circle NW,452.0,38.890539,-77.049383
4,C & O Canal & Wisconsin Ave NW,127.0,38.903750,-77.062690


In [13]:
join = stations.merge(summary, left_on="station_name", right_on="start", how="right").dropna()

In [15]:
geojson = {"type": "FeatureCollection"}
features = []

In [16]:
last_station = ""
feature = {}

for i in range(len(join)):
    station_name = join.loc[i, "station_name"]
    
    if (last_station != station_name):
        last_station = station_name
        features.append(feature)
        
        feature = {
            "type": "Feature", 
            "properties": {
                "station_name": station_name, 
                "to": {
                    join.loc[i, "end"]: int(join.loc[i, "trips"])
                }
            }, 
            "geometry": {"type": "Point", 
                         "coordinates": [join.loc[i, "lng"], join.loc[i, 'lat']]
                        }
        }
    else:
        feature["properties"]["to"][join.loc[i, "end"]] = int(join.loc[i, "trips"])
        
    

In [17]:
geojson["features"] = features[1:]

In [18]:
geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'station_name': 'Constitution Ave & 2nd St NW/DOL',
    'to': {'Constitution Ave & 2nd St NW/DOL': 482,
     '15th St & Constitution Ave NW': 165,
     'Jefferson Dr & 14th St SW': 156,
     'Lincoln Memorial': 148,
     'Henry Bacon Dr & Lincoln Memorial Circle NW': 124,
     '1st & M St NE': 120,
     'Smithsonian-National Mall / Jefferson Dr & 12th St SW': 116,
     '4th St & Madison Dr NW': 97,
     '15th St & Pennsylvania Ave NW': 94,
     '4th St & K St NW': 68,
     '4th & C St SW': 59,
     '1st & K St NE': 54,
     '17th St & Independence Ave SW': 53,
     '3rd & M St NE': 48,
     '10th & E St NW': 47,
     'Columbus Circle / Union Station': 47,
     'Metro Center / 12th & G St NW': 47,
     '5th St & Massachusetts Ave NW': 46,
     '19th St & Constitution Ave NW': 45,
     'New Hampshire Ave & T St NW': 45,
     '17th & G St NW': 44,
     '21st St & Constitution Ave NW': 43,
     '14th & D St N

In [19]:
with open('data.txt', 'w') as outfile:
    json.dump(geojson, outfile)